In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[2]:

In [126]:
import datetime
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from tranco import Tranco
import json
import waybackpy
import glob, os, os.path
from multiprocessing import Manager

from simpletransformers.classification import ClassificationModel
from simpletransformers.config.model_args import ClassificationArgs

In [145]:
#https://github.com/marty90/priv-accept/blob/7c87be9441a6425160611d7fd91fd1e4ae7bceaf/priv-accept.py

def click_banner(driver):
    accept_words_list = set()
    for w in open("accept_words_languages.txt", "r", encoding="utf-8").read().splitlines():
        if not w.startswith("#") and not w == "":
            accept_words_list.add(w)

    sleep(20)
    content = driver.find_elements(By.CSS_SELECTOR, GLOBAL_SELECTOR)
    for element in content:
        try:
            if element.text.lower().strip(" ✓›!\n") in accept_words_list:
                print(element.text)
                print("and here")
                #WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, driver.get(element)))).click()
                element.click()
                return {'successful': True, 'error': 'none'}
        except Exception as e:
            print("Failed", e)
            return {'successful': False, 'error': 'click_error'}
    return {'successful': False, 'error': 'no_accept_button' }

In [146]:
def add_iframe(driver):
    new_candidates = []
    try:
        temp = driver.find_elements(By.XPATH, '//iframe')
        for element in temp:
            if element.size['width'] > 10 and element.size['height'] > 10:
                new_candidates.append(element)
    except Exception as err:
        print(err)
    return new_candidates

In [147]:
def filter_candidates(candidates):
    try:
        for index, c in reversed(list(enumerate(candidates))):
            text = c.text
            if c.tag_name in ["style", "ul", "il", "a", "svg", "button"]:  # Element that can't be a cookie dialog
                # print('Candidate popped = ' + c.tag_name)
                candidates.pop(index)
            elif c.tag_name == 'iframe':  # iframes
                pass
            elif len(text) < 50: # too small
                # print('Candidate popped too small text')
                candidates.pop(index)
    except Exception as err:
        print(err)
    
    # Filter out elements that are a father of another element (or same element)
    i = 0
    while i < len(candidates):
        for index, c in enumerate(candidates):
            if index == i:
                pass
            elif candidates[i].get_attribute('outerHTML') in c.get_attribute('outerHTML'):
                candidates.pop(index)
                # print('Candidate popped, is father of other element')
                if index <= i:
                    i -= 1
        i += 1

    return candidates


In [148]:
def finetune_element(element, type_ifr='normal'):
    try:
        try: # Check for iframes
            iframe = element.find_element(By.XPATH, './/iframe')
        except:
            iframe = False

        if element.tag_name == 'iframe':
            pass
        elif iframe and type_ifr == 'normal':
            # print(iframe)
            element = iframe
        else:
            last_element = False
            chi = element.find_elements(By.XPATH, "./*")
            while not last_element and len(chi) <= 2:
                # print(chi)
                if len(chi) == 0:  # List is empty: done
                    last_element = True
                    break
                elif len(chi) == 1: # List contains 1 child: go to child
                    element = chi[0]
                    # print(element)
                    chi = element.find_elements(By.XPATH, "./*")
                else: # List contains 2 children: go to child if the other child is empty
                    if chi[0].get_attribute('innerHTML') == "" or chi[0].tag_name == 'head':
                        chi = chi[1].find_elements(By.XPATH, "./*")
                    elif chi[1].get_attribute('innerHTML') == "":
                        chi = chi[0].find_elements(By.XPATH, "./*")
                    else:
                        last_element = True

    except Exception as err:
        print(traceback.format_exc())
        print(err)

    return element

In [149]:
def do_Frames(driver):
    date_of_reject = False
    
    candidates = []
    candidates.extend(add_iframe(driver))
    print(candidates)
    candidates = filter_candidates(candidates)
    print(candidates)
    cookie_dialogue_found = False
    
    if not candidates:
        print("No candidates found")
        pass
    
    else:
        model_dialog = ClassificationModel("xlmroberta", ML_cookie_dialogue, args={"silent": True}, use_cuda=False)
        index = 0
        while not date_of_reject and index < len(candidates):
                c = candidates[index]
                index += 1
                try:
                    driver.switch_to.frame(c)
                    sleep(0.5)
                    c = driver.find_element(By.XPATH, '/*')
                    c = finetune_element(c, 'iframe')
                    CD_text = c.text
                    try:
                         if (c.size['width'] > 10 or c.size['height'] > 10) and CD_text and len(CD_text) >= 20:
                                        # Check for cookie dialog with trained model
                                        with lock:
                                            prediction_CD, raw_outputs = model_dialog.predict(
                                                [CD_text.lower().replace('\n', ' ').replace(';', ',')])

                                        print("Prediction made, result: {} - {}".format(prediction_CD[0], ''.join(
                                            [x for i, x in enumerate(CD_text.replace('\n', ' ')) if i < 124])))
                                        short_text = ''.join(
                                            [x for i, x in enumerate(CD_text.replace('\n', ' ')) if i < 1020])

                                        # Add basic cookie dialog info to database
                                        predictions.append([visit_id, "cookie dialog", c.tag_name, short_text, prediction_CD[0]])

                                        if prediction_CD[0] == "True":
                                            cookie_dialogue_found = True

                                            cookie_elements = c.find_elements(By.XPATH,
                                                                              ".//button | .//a | .//span | .//svg")
                                            with lock:
                                                all_texts = []
                                                # Laadt model voor button classifier in
                                                model_buttons = ClassificationModel("xlmroberta", ML_cookie_buttons,
                                                                                    args={"silent": True},
                                                                                    use_cuda=False)
                                                for el in cookie_elements:
                                                    text = el.text

                                                    if len(text) > 0 and text not in all_texts: # Check als nog niet voorgekomen en tekst bevat
                                                        all_texts.append(text)

                                                        prediction_B, raw_outputs2 = model_buttons.predict(
                                                            [text.lower().replace("\n", " ")])
                                                        print("Prediction made, result: {} - {}".format(prediction_B[0],
                                                                    ''.join([x for index, x in enumerate(text.replace('\n', ' ')) if index < 124])))
                                                        predictions.append([visit_id, "button", el.tag_name, text, prediction_B[0]])

                                                        if button_type == "REJECT":
                                                            return True
                    except Exception as err:
                        print(err)
                except Exception as err:
                    print(err)
        return False

In [150]:
options = FirefoxOptions()
#options.headless = True
service = FirefoxService(r'C:\Program Files\Mozilla Firefox\geckodriver.exe')
driver = webdriver.Firefox(service=service, options=options)

KeyboardInterrupt: 

In [144]:
url1 = "https://www.free.fr"
driver.get(url1)
sleep(10)
abc = do_Frames(driver)
print(abc)

[<selenium.webdriver.remote.webelement.WebElement (session="9a4e81b6-b907-475d-bcf3-ca0550c1aa77", element="271d4240-7f17-4b9f-a837-e5a98c860afa")>, <selenium.webdriver.remote.webelement.WebElement (session="9a4e81b6-b907-475d-bcf3-ca0550c1aa77", element="5cae856e-2e6f-4a5b-ae8a-5867c2395757")>, <selenium.webdriver.remote.webelement.WebElement (session="9a4e81b6-b907-475d-bcf3-ca0550c1aa77", element="27ce8f0c-a3eb-4566-911f-c5f03698a66c")>, <selenium.webdriver.remote.webelement.WebElement (session="9a4e81b6-b907-475d-bcf3-ca0550c1aa77", element="da2dbe7f-1091-4b7f-8e3b-7dca9f4ce3ef")>, <selenium.webdriver.remote.webelement.WebElement (session="9a4e81b6-b907-475d-bcf3-ca0550c1aa77", element="b3ca9b86-18c5-4baa-b46d-fe8072f2c76b")>, <selenium.webdriver.remote.webelement.WebElement (session="9a4e81b6-b907-475d-bcf3-ca0550c1aa77", element="d1484b82-aad5-40e1-ac2c-31f1433e82bf")>, <selenium.webdriver.remote.webelement.WebElement (session="9a4e81b6-b907-475d-bcf3-ca0550c1aa77", element="2ce7

[<selenium.webdriver.remote.webelement.WebElement (session="9a4e81b6-b907-475d-bcf3-ca0550c1aa77", element="b3ca9b86-18c5-4baa-b46d-fe8072f2c76b")>, <selenium.webdriver.remote.webelement.WebElement (session="9a4e81b6-b907-475d-bcf3-ca0550c1aa77", element="2ce71fe0-817e-463c-ae4f-df895441151d")>, <selenium.webdriver.remote.webelement.WebElement (session="9a4e81b6-b907-475d-bcf3-ca0550c1aa77", element="0e605642-b65c-41ef-a440-efa4f34500ff")>, <selenium.webdriver.remote.webelement.WebElement (session="9a4e81b6-b907-475d-bcf3-ca0550c1aa77", element="d9c14271-ac12-419f-b990-c6dcfd0ba49f")>, <selenium.webdriver.remote.webelement.WebElement (session="9a4e81b6-b907-475d-bcf3-ca0550c1aa77", element="335bd5a2-6416-42f0-9df0-04ae1f32a5fa")>, <selenium.webdriver.remote.webelement.WebElement (session="9a4e81b6-b907-475d-bcf3-ca0550c1aa77", element="acac6242-6b74-48c8-8805-026c163373b9")>, <selenium.webdriver.remote.webelement.WebElement (session="9a4e81b6-b907-475d-bcf3-ca0550c1aa77", element="a375

Message: Unable to locate frame for element: [object HTMLDivElement]
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:180:5
NoSuchFrameError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:423:5
switchToFrame@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:556:15
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:165:31

Message: Unable to locate frame for element: [object HTMLDivElement]
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:180:5
NoSuchFrameError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:423:5
switchToFrame@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:556:15
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:165:31

Message:

Message: Unable to locate frame for element: [object HTMLDivElement]
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:180:5
NoSuchFrameError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:423:5
switchToFrame@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:556:15
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:165:31

Message: Unable to locate frame for element: [object HTMLDivElement]
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:180:5
NoSuchFrameError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:423:5
switchToFrame@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:556:15
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:165:31

Message:

False


In [8]:
# Initializing tranco-list.eu list.
t = Tranco(cache=True, cache_dir='.tranco')
#latest_list = t.list()

# Get tranco list for the specific date
date_list = t.list(date=DATE_TRANCO_LIST)

In [14]:
def to_integer(dt_time):
    return 10000*dt_time.year + 100*dt_time.month + dt_time.day

In [15]:
# Filter the list to what we are looking for.
french_websites = [web for web in date_list.list if '.fr' in web]
french_websites_www = [f'{website}' for website in french_websites]

first_three_websites = french_websites_www[0:NUMBER_OF_WEBSITES]

def waybackify_url(url, closest_timestamp):
    return f'https://web.archive.org/web/{closest_timestamp}/{url}'

loist = {}

import time 


for i in first_three_websites:
    loist2 = []
    start_date = START_DATE_FRANCE
    while start_date <= END_DATE_FRANCE:
        url = waybackify_url(i, to_integer(start_date))
        loist2.append((url, (start_date.year, start_date.month)))
        start_date += delta
    loist[i] = loist2

def to_integer(dt_time):
    return 10000*dt_time.year + 100*dt_time.month + dt_time.day

#for el in list(base):
#    time.mktime(datetime.datetime.strptime(el, "%d/%m/%Y").timetuple())
    
print(loist)
#print(first_three_websites)

{'free.fr': [('https://web.archive.org/web/20200714/free.fr', (2020, 7)), ('https://web.archive.org/web/20200814/free.fr', (2020, 8)), ('https://web.archive.org/web/20200914/free.fr', (2020, 9)), ('https://web.archive.org/web/20201015/free.fr', (2020, 10)), ('https://web.archive.org/web/20201115/free.fr', (2020, 11)), ('https://web.archive.org/web/20201216/free.fr', (2020, 12))], 'google.fr': [('https://web.archive.org/web/20200714/google.fr', (2020, 7)), ('https://web.archive.org/web/20200814/google.fr', (2020, 8)), ('https://web.archive.org/web/20200914/google.fr', (2020, 9)), ('https://web.archive.org/web/20201015/google.fr', (2020, 10)), ('https://web.archive.org/web/20201115/google.fr', (2020, 11)), ('https://web.archive.org/web/20201216/google.fr', (2020, 12))], 'amazon.fr': [('https://web.archive.org/web/20200714/amazon.fr', (2020, 7)), ('https://web.archive.org/web/20200814/amazon.fr', (2020, 8)), ('https://web.archive.org/web/20200914/amazon.fr', (2020, 9)), ('https://web.arch

In [51]:
def go_over_months(website):
    # create object from passed website
    wayback_obj = waybackpy.Url(website, USER_AGENT).save()
    print(website)
    print(wayback_obj)

    # list of urls for the website for each month
    urls_of_dates = []

    # iterating between 01-07-2019 to 25-03-25 by jumps of 31 days 
    start_date = START_DATE_FRANCE

    # append list of urls using wayback obect wuth specific date to interact with the API.
    while start_date <= END_DATE_FRANCE:
        # sometimes the API fails so in that case we simply try again
        try:
            urls_of_dates.append((wayback_obj.near(year=start_date.year, month=start_date.month).archive_url, (start_date.year, start_date.month)))
            print(urls_of_dates, "Success.")
            0# go to the next month
            start_date += delta
            # the API is rather slow so we need to give some time to rest (:
            sleep(3)
        except Exception:
            print(Exception)
            continue
    return urls_of_dates

In [52]:
def go_over_websites(websites_list):
    # Create 2D list of all websites urls for each month 
    get_list = {}
    for url in websites_list:

        # 2D list
        get_list[url] = go_over_months(url)
        print(get_list)
        
    return get_list

In [40]:
user_option = ['NOTHING', 'ACCEPT', 'REJECT', 'MANAGE']
GLOBAL_SELECTOR = "a, button, div, span, form, p"
NUMBER_OF_WEBSITES = 3

USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:105.0) Gecko/20100101 Firefox/105.0"
START_DATE_FRANCE = datetime.date(2020,7, 14)
END_DATE_FRANCE = datetime.date(2020, 12, 25)
delta = datetime.timedelta(days=31)

ML_cookie_dialogue = "./models/dialogue_model0"
ML_cookie_buttons = "./models/button_model0"

predictions = Manager().list([])

DATE_TRANCO_LIST='2022-10-01'

In [42]:
dictory_of_urls = go_over_websites(first_three_websites)

free.fr
https://web.archive.org/web/20230413183409/https://www.free.fr/freebox/
[('https://web.archive.org/web/20170616092025/http://www2004.free.fr/', (2020, 7))] Success.
[('https://web.archive.org/web/20170616092025/http://www2004.free.fr/', (2020, 7)), ('https://web.archive.org/web/20170616092025/http://www2004.free.fr/', (2020, 8))] Success.
[('https://web.archive.org/web/20170616092025/http://www2004.free.fr/', (2020, 7)), ('https://web.archive.org/web/20170616092025/http://www2004.free.fr/', (2020, 8)), ('https://web.archive.org/web/20170616092025/http://www2004.free.fr/', (2020, 9))] Success.
[('https://web.archive.org/web/20170616092025/http://www2004.free.fr/', (2020, 7)), ('https://web.archive.org/web/20170616092025/http://www2004.free.fr/', (2020, 8)), ('https://web.archive.org/web/20170616092025/http://www2004.free.fr/', (2020, 9)), ('https://web.archive.org/web/20170616092025/http://www2004.free.fr/', (2020, 10))] Success.
[('https://web.archive.org/web/20170616092025/htt

In [ ]:
print(dictory_of_urls)

In [ ]:
with open("{}_all_urls.txt".format("Urls"), 'w') as outfile:
                json.dump(dictory_of_urls, outfile, indent = 4)

In [103]:
options = FirefoxOptions()
options.headless = True
service = FirefoxService(r'C:\Program Files\Mozilla Firefox\geckodriver.exe')
driver = webdriver.Firefox(service=service, options=options)



In [63]:
def main():
    
    # Collect cookies for every month, write to two JSON files: before click and after ACCEPT ALL,
    # and return the number of cookies.
    for website in loist:
        
        website_name = website
        
        for url in loist[website]:
            
            print(website, url[1])
            
            driver.get(url[0])
            sleep(30)
            correct_date = do_Frames(driver)
            sleep(1)
            
            print(correct_date)
            
            # create two JSON files
            #with open("{}_cookies_no_click.json".format(website_name), 'a') as outfile:
            #    json.dump((url[1], cookies_before), outfile, indent = 4)
            #with open("{}_cookies_after_click.json".format(website_name), 'a') as outfile:
            #    json.dump((url[1], cookies_after), outfile, indent = 4)

main()
driver.quit()

free.fr (2020, 7)
No candidates found
None
free.fr (2020, 8)
No candidates found
None
free.fr (2020, 9)
No candidates found
None
free.fr (2020, 10)
No candidates found
None
free.fr (2020, 11)
No candidates found
None
free.fr (2020, 12)
No candidates found
None
google.fr (2020, 7)
No candidates found
None
google.fr (2020, 8)
No candidates found
None
google.fr (2020, 9)
No candidates found
None
google.fr (2020, 10)
No candidates found
None
google.fr (2020, 11)
No candidates found
None
google.fr (2020, 12)
No candidates found
None
amazon.fr (2020, 7)
No candidates found
None
amazon.fr (2020, 8)
No candidates found
None
amazon.fr (2020, 9)
No candidates found
None
amazon.fr (2020, 10)
No candidates found
None
amazon.fr (2020, 11)
No candidates found
None
amazon.fr (2020, 12)
No candidates found
None


In [104]:
url1 = "https://www.free.fr"
driver.get(url1)
sleep(10)
abc = do_Frames(driver)
print(abc)

Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good
Size is good

In [ ]:
mydir = r"V:\Uni\Thesis\Code\Thesis"
filelist = glob.glob(os.path.join(mydir, "*.JSON"))
for f in filelist:
    os.remove(f)


In [41]:
for website in dictory_of_urls:
       website_name = website
       print(website)
       for url in dictory_of_urls[website]:
           print(url[1])

dictorio = {}
dictory_of_urls
print(dictory_of_urls)


NameError: name 'dictory_of_urls' is not defined